In [ ]:
# ensure that any code changes are immediately reflected
%reload_ext autoreload
%autoreload 2

## Initialize Three Users

In [ ]:
from mostlyai import MostlyAI
base_url = 'https://dev.dev.mostlylab.com/'
# Test User 1
mostly_t1 = MostlyAI(api_key='mostly-765c7f96e0a8f261b55248f8672e46dd7281ef90219328ce071b5a93fbb65edf', base_url=base_url)
# Test User 2
mostly_t2 = MostlyAI(api_key='mostly-fc5d7602dba784ce687d775b72c12d2e93407fba9731144145b85e0548ac1e97', base_url=base_url)
# Test User 3
mostly_t3 = MostlyAI(api_key='mostly-0dd57f8b8cdb7485310aadead0439c48beea8956273614102e8bf7950bc3f02e', base_url=base_url)

## Basic Usage

### User 1 trains a Generator

In [ ]:
g = mostly_t1.train('https://raw.githubusercontent.com/mostly-ai/public-demo-data/dev/titanic/titanic.csv')

In [ ]:
g

In [ ]:
# Open the created generator in a web browser 
g.open()

### User 1 shares the Generator with User 2 and User 3

In [ ]:
mostly_t1.share(g, 'test2@mostly.ai')
mostly_t1.share(g, 'test3@mostly.ai')

### User 2 generates a Synthetic Dataset

In [ ]:
sd = mostly_t2.generate(g.id, size=1000)

### User 2 consumes the Synthetic Dataset

In [ ]:
sd.data()

In [ ]:
sd

### User 3 generates a Seeded Synthetic Dataset

In [ ]:
# prepare a seed DataFrame
import pandas as pd
import numpy as np
seed_df = pd.DataFrame({'survived': [1] * 10_000, 'age': np.random.randint(5, 18, 10_000)})
seed_df

In [ ]:
sd3 = mostly_t3.generate(g.id, seed=seed_df)

In [ ]:
sd3.data()

## Multi-Table

### User 1 trains a 3-table setup from pandas

In [ ]:
import pandas as pd
df_players = pd.read_csv('https://github.com/mostly-ai/public-demo-data/raw/dev/baseball/players.csv.gz')
df_fielding = pd.read_csv('https://github.com/mostly-ai/public-demo-data/raw/dev/baseball/fielding.csv.gz')
df_batting = pd.read_csv('https://github.com/mostly-ai/public-demo-data/raw/dev/baseball/batting.csv.gz')
df_players

In [ ]:
g = mostly_t1.train(config=
    {
        "name": "baseball",
        "tables": [
            {
                "name": "players", 
                "data": df_players, 
                "primaryKey": "id",
                "modelConfiguration": {"maxTrainingTime": 1}
            },
            {
                "name": "fielding",
                "data": df_fielding, 
                "foreignKeys": [{"column": "players_id", "referencedTable": "players", "isContext": True}],
                "modelConfiguration": {"maxTrainingTime": 1}
            },
            {
                "name": "batting",
                "data": df_batting, 
                "foreignKeys": [{"column": "players_id", "referencedTable": "players", "isContext": True}],
                "modelConfiguration": {"maxTrainingTime": 1}
            }
        ]
    }
)

## Database Connectivity

### User 1 creates a Connector to a Postgres Database

In [ ]:
import os
c = mostly_t1.connect({
    "accessType": "SOURCE",
    "type": "POSTGRES",
    "config": {
        "host": "postgres-test.cppq7davohkj.eu-central-1.rds.amazonaws.com",
        "username": "postgres",
        "database": "berka_original",
    },
    "secrets": {
        "password": pwd
    }
})

### User 1 shares Connector with User 2

In [ ]:
mostly_t1.share(c.id, 'test2@mostly.ai')

### User 2 uses Connector to train a Generator

In [ ]:
c2 = mostly_t2.connectors.get(c.id)
c2

In [ ]:
c2.locations()

In [ ]:
c2.locations("berka")

In [ ]:
g = mostly_t2.train(config=
    {
        "name": "BERKA",
        "tables": [
            {
                "name": "accounts", 
                "sourceConnectorId": str(c2.id),
                "location": "berka.account",
                "primaryKey": "account_id"
            }
        ]
    }
)

In [ ]:
sd = mostly_t2.generate(g)

In [ ]:
sd.data()